# Qwen Sentiment 模型演示

本笔记演示如何在本地环境中加载微调后的 Qwen 情感模型，并对新闻文本进行情感打分。运行前请先完成以下准备：

- 在 Conda/venv 中安装 `torch`, `transformers`, `peft`, `pandas` 等依赖。
- 将基础模型目录和微调后的 LoRA 权重目录放在本地磁盘，并记录实际路径。
- 如果使用 CPU 推理，请确认显存/内存允许（可将下方的 `torch_dtype` 调整为 `torch.float32` 并强制 `device_map={"": "cpu"}`）。


In [ ]:
from pathlib import Path

# TODO: 将下面的路径替换为你本地的实际目录
BASE_MODEL_DIR = Path(r"C:\\Users\\nings\\Downloads\\居丽叶简历项目3：股票投资顾问Agent\\Finance\\Qwen")
FINETUNED_MODEL_DIR = Path(r"C:\\Users\\nings\\Downloads\\居丽叶简历项目3：股票投资顾问Agent\\Finance\\qwen_sentiment_model")

if not BASE_MODEL_DIR.exists():
    raise FileNotFoundError(f"基础模型目录不存在: {BASE_MODEL_DIR}")

if not FINETUNED_MODEL_DIR.exists():
    raise FileNotFoundError(f"LoRA 模型目录不存在: {FINETUNED_MODEL_DIR}")


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

TORCH_DTYPE = torch.float16  # 如需 CPU 推理，可改为 torch.float32
DEVICE_MAP = "auto"          # CPU 推理可改为 {"": "cpu"}


def load_sentiment_model(base_model_dir: Path, finetuned_model_dir: Path):
    """从本地路径加载基础模型与 LoRA 权重。"""
    print("正在加载 tokenizer …")
    tokenizer = AutoTokenizer.from_pretrained(finetuned_model_dir)

    print("正在加载基础模型 …")
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_dir,
        torch_dtype=TORCH_DTYPE,
        device_map=DEVICE_MAP,
    )

    print("正在合并 LoRA 权重 …")
    model = PeftModel.from_pretrained(base_model, finetuned_model_dir)
    model.eval()
    return model, tokenizer


def create_prompt(text: str, stock_symbol: str = "STOCK") -> str:
    system_prompt = (
        "Forget all your previous instructions. You are a financial expert with stock "
        "recommendation experience. Based on a specific stock, score for range from 1 to 5, "
        "where 1 is negative, 2 is somewhat negative, 3 is neutral, 4 is somewhat positive, "
        "5 is positive. 1 summarized news will be passed in each time, you will give score in "
        "format as shown below in the response from assistant."
    )

    few_shot_examples = (
        "User: News to Stock Symbol -- AAPL: Apple (AAPL) increase 22%\n"
        "Assistant: 5\n\n"
        "User: News to Stock Symbol -- AAPL: Apple (AAPL) price decreased 30%\n"
        "Assistant: 1\n\n"
        "User: News to Stock Symbol -- AAPL: Apple (AAPL) announced iPhone 15\n"
        "Assistant: 4\n\n"
    )

    current_query = f"User: News to Stock Symbol -- {stock_symbol}: {text}\nAssistant:"

    return f"System: {system_prompt}\n\n{few_shot_examples}{current_query}"


def predict_sentiment(model, tokenizer, text: str, stock_symbol: str = "STOCK") -> int | None:
    prompt = create_prompt(text, stock_symbol)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=5,
            do_sample=False,
            temperature=0.1,
            pad_token_id=tokenizer.eos_token_id,
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    assistant_response = generated_text.split("Assistant:")[-1].strip()

    try:
        sentiment_score = int(assistant_response.split()[0])
        if 1 <= sentiment_score <= 5:
            return sentiment_score
    except Exception:
        pass

    return None


In [ ]:
model, tokenizer = load_sentiment_model(BASE_MODEL_DIR, FINETUNED_MODEL_DIR)
model


In [ ]:
sample_news = [
    ("Apple reported strong quarterly earnings with revenue growth of 15%", "AAPL"),
    ("Apple faces supply chain disruptions and production delays", "AAPL"),
    ("Tesla delivers record number of vehicles in Q4", "TSLA"),
]

results = []
for text, symbol in sample_news:
    score = predict_sentiment(model, tokenizer, text, symbol)
    results.append({
        "stock_symbol": symbol,
        "news": text,
        "predicted_sentiment": score,
    })

results


In [ ]:
import pandas as pd

pd.DataFrame(results)


## 下一步

- 如需批量预测，可将上面的 `sample_news` 替换为来自 CSV/DataFrame 的真实新闻数据。
- 风险模型的使用方式类似，只需将提示函数字段调整为风险评分说明，并加载 `qwen_risk_model` 权重。
- 推理结束后记得释放 GPU 显存，可执行 `del model; torch.cuda.empty_cache()`。
